In [18]:
import string
from datasets import load_dataset
from rapidfuzz import fuzz
from tqdm import tqdm

In [25]:
def subset_check(small, big):
    intersection = small.intersection(big)

    return len(intersection) == len(small)

In [7]:
ds = load_dataset("allenai/scirepeval", "search")

Generating validation split: 100%|██████████| 75382/75382 [00:21<00:00, 3560.70 examples/s]


In [52]:
total = 0
title_search_count = 0
doc_search_count = 0
title_search_count_2 = 0
to_check = []
for line in tqdm(ds["train"]):
    query = line["query"].replace(' AND ', " ").replace(" OR ", " ").lower().strip(string.punctuation).strip().replace("-", " ").replace('"', "")
    query_tokens = set([item.strip(string.punctuation).strip() for item in query.split(" ")])

    candidates = line["candidates"]

    for cand in candidates:
        flag = False
        if cand["score"] < 1: continue
        title = cand["title"].lower().strip(string.punctuation).strip().replace("-", " ")
        title_tokens = set([item.strip(string.punctuation) for item in title.split(" ")])
        abstract = cand["abstract"].lower().strip(string.punctuation).strip()

        doc = title + ". " +abstract

        # if fuzz.ratio(query, title) >= 90: title_search_count += 1
        if query in title: 
            title_search_count += 1
            flag = True
        if query in doc or doc in query: 
            doc_search_count += 1
            flag = True
        if subset_check(query_tokens, title_tokens): 
            title_search_count_2 += 1
            flag = True


        if not flag: to_check.append([query, title, abstract])
        # if fuzz.partial_ratio(query, title) >= 90: partial_title_search_count += 1
        # if fuzz.partial_ratio(query, title + ". " + abstract) >= 90: partial_doc_search_count += 1
        total += 1

100%|██████████| 399878/399878 [02:03<00:00, 3230.67it/s]


In [ ]:
title_search_count / total, title_search_count_2 / total

In [43]:
title_search_count / total, title_search_count_2 / total

(0.441453586686601, 0.5316137853482755)

In [51]:
to_check[112]

['barium enema in colitis',
 "crohn's disease in a developing african mission hospital: a case report",
 "backgrounda case is reported of innocuous intestinal obstruction requiring surgical intervention that was confirmed to be crohn's disease histopathologically in a resource constrained rural mission hospital in cameroon.case presentationa 70 year man of kumbo origin from northwest region of cameroon with a history of crampy right lower quadrant abdominal pain, non bloody, non mucoid diarrhea alternating with constipation presented to my institution. abdominal examination of the patient revealed an ill defined mass in the right iliac fossa and visible peristalsis. an abdominal computed tomographic scan and barium enema study confirmed a complex ascending colonic and cecal tumor. the patient underwent exploratory laparotomy. the intraoperative finding was a huge complex inflammatory mass involving the cecum, terminal ileum, and sigmoid colon. he subsequently had sigmoidectomy with end

In [28]:
partial_title_search_count / total

0.5414659391492135

In [29]:
partial_doc_search_count / total

0.5404117203572844

In [48]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'